In [1]:
import psycopg2

conn = psycopg2.connect(
    host="127.0.1.1",
    port="5433",
    dbname="yugabyte",
    user="yugabyte",
    password="yugabyte"
)

cursor = conn.cursor()
cursor.execute("SELECT version();")
print(cursor.fetchone())

conn.close()

('PostgreSQL 11.2-YB-2024.2.3.0-b0 on x86_64-pc-linux-gnu, compiled by clang version 17.0.6 (https://github.com/yugabyte/llvm-project.git 9b881774e40024e901fc6f3d313607b071c08631), 64-bit',)


In [34]:
conn = psycopg2.connect(
    host="127.0.1.1",
    port="5433",
    dbname="yugabyte",
    user="yugabyte",
    password="yugabyte"
)
cursor = conn.cursor()
cursor.execute("select * from cd.bookings")
print(cursor.scroll)

<built-in method scroll of psycopg2.extensions.cursor object at 0x7f34d440dd50>


In [35]:
c=conn.cursor()
c.execute("SELECT * FROM pg_database;")
for i in c:
    print(i)
c=conn.cursor()
conn.close()

('template1', 10, 6, 'C', 'en_US.UTF-8', True, True, -1, 0, '0', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
('template0', 10, 6, 'C', 'en_US.UTF-8', True, False, -1, 0, '0', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
('postgres', 10, 6, 'C', 'en_US.UTF-8', False, True, -1, 0, '0', '1', 1663, None)
('yugabyte', 10, 6, 'C', 'en_US.UTF-8', False, True, -1, 0, '0', '1', 1663, None)
('system_platform', 10, 6, 'C', 'en_US.UTF-8', False, True, -1, 0, '0', '1', 1663, None)


In [45]:
import psycopg2

conn = psycopg2.connect(
    host="127.0.1.1",
    port="5433",
    dbname="yugabyte",
    user="yugabyte",
    password="yugabyte"
)
c = conn.cursor()

# Drop table safely
drop = """DROP TABLE IF EXISTS employee;"""
c.execute(drop)

# Create table with sharding
employee_table = """CREATE TABLE employee (
  empid SERIAL PRIMARY KEY,
  empname VARCHAR(45),
  department VARCHAR(45),
  salary INT
) SPLIT INTO 8 TABLETS;"""
c.execute(employee_table)

# Describe table structure
describe_query = """SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'employee';"""
c.execute(describe_query)
for i in c:
  print(i)
# Fetch & print table structure
print(c.fetchall())

conn.commit()
conn.close()

('empid', 'integer')
('salary', 'integer')
('empname', 'character varying')
('department', 'character varying')
[]


In [54]:
import psycopg2

# Establish the connection
conn = psycopg2.connect(
    host="127.0.1.1",
    port="5433",
    dbname="yugabyte",
    user="yugabyte",
    password="yugabyte"
)

# Create a cursor object
c = conn.cursor()

# SQL insert with conflict handling: update if empid already exists
insert_data = """
INSERT INTO EMPLOYEE (empid, empname, department, salary)
VALUES (%s, %s, %s, %s)
ON CONFLICT (empid)
DO UPDATE SET
    empname = EXCLUDED.empname,
    department = EXCLUDED.department,
    salary = EXCLUDED.salary
"""

# List of employee data
data = [
    (1, 'John Doe', 'HR', 50000),
    (2, 'Jane Smith', 'Finance', 60000),
    (3, 'Robert Brown', 'Engineering', 75000),
    (4, 'Emily Davis', 'Marketing', 55000),
    (5, 'Michael Johnson', 'Sales', 58000),
    (6, 'Sarah Wilson', 'Customer Support', 45000),
    (7, 'Daniel Martinez', 'IT', 72000),
    (8, 'Sophia Lopez', 'Operations', 53000),
    (9, 'David Clark', 'Legal', 69000),
    (10, 'Linda Garcia', 'Admin', 48000)
]

# Insert or update all rows
c.executemany(insert_data, data)

# Commit the transaction
conn.commit()

# Close the cursor and connection
c.close()
conn.close()


In [56]:
import psycopg2

# Establish the connection
conn = psycopg2.connect(
    host="127.0.1.1",
    port="5433",
    dbname="yugabyte",
    user="yugabyte",
    password="yugabyte"
)

# Create a cursor object
c = conn.cursor()
employee_table_details="""select * from employee order by empid asc"""
c.execute(employee_table_details)
employee_table=c.fetchall()
for e in employee_table:
    print(e)
conn.close()

(1, 'John Doe', 'HR', 50000)
(2, 'Jane Smith', 'Finance', 60000)
(3, 'Robert Brown', 'Engineering', 75000)
(4, 'Emily Davis', 'Marketing', 55000)
(5, 'Michael Johnson', 'Sales', 58000)
(6, 'Sarah Wilson', 'Customer Support', 45000)
(7, 'Daniel Martinez', 'IT', 72000)
(8, 'Sophia Lopez', 'Operations', 53000)
(9, 'David Clark', 'Legal', 69000)
(10, 'Linda Garcia', 'Admin', 48000)
